In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [ ]:
import json
from satori.rtm.client import make_client, SubscriptionMode

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=10   # number of inputs
    beta=np.zeros(n)
    beta[1]=1   ## real parameters
    beta[-1]=1
    beta.shape=(n,1)
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    lastPrice=float(json.loads(json.dumps(messagebox.pop(), ensure_ascii=False))['price']) # get that first price
    proxy=0
    j=1 # j = nombre de rendements
    while True:
        proxies=np.empty(0) # x of the RLS algorithm
        while proxies.size<n:            
            if messagebox:
                message=messagebox.pop()
                message=json.dumps(message, ensure_ascii=False)
                message=json.loads(message)
                proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
                lastPrice=float(message['price'])
                proxies=np.append(proxies, proxy)
                j+=1
                print(proxies, proxies.size)
            time.sleep(1)
        y=float(proxies.dot(beta))+0.1*np.random.rand(1)[0] # slide 31: y =x^T beta +w / np.random.rand(1)[0] gives number 0<n<1
        message=np.array2string(np.append([i,y], proxies),separator=",") 
        producer.send('test2', message)
        i=i+1
        print(message)

Connected to Satori RTM!
(array([  2.81256965e-09]), 1)
(array([  2.81256965e-09,   1.62454768e-07]), 2)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07]), 3)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07]), 4)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07,   2.44802395e-07]), 5)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07,   2.44802395e-07,   2.44802720e-07]), 6)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07,   2.44802395e-07,   2.44802720e-07,
         2.44802720e-07]), 7)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07,   2.44802395e-07,   2.44802720e-07,
         2.44802720e-07,   2.45052097e-07]), 8)
(array([  2.81256965e-09,   1.62454768e-07,   2.06308365e-07,
         2.44209728e-07,   2.44802395e-07,   2.44802720e-07,
         2.44802720e-07,   2.45052097e-07,   2.4527

[  5.00000000e+00,  8.85215332e-02,  2.91306332e-06,  3.06593595e-06,
   3.07354420e-06,  3.07464253e-06,  3.09346235e-06,  3.09486083e-06,
   3.13308460e-06,  3.13312412e-06,  3.14112148e-06,  3.14180589e-06]
(array([  3.14775780e-06]), 1)
(array([  3.14775780e-06,   3.17497273e-06]), 2)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06]), 3)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06,
         3.18412124e-06]), 4)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06,
         3.18412124e-06,   3.19068691e-06]), 5)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06,
         3.18412124e-06,   3.19068691e-06,   3.19162296e-06]), 6)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06,
         3.18412124e-06,   3.19068691e-06,   3.19162296e-06,
         3.32251561e-06]), 7)
(array([  3.14775780e-06,   3.17497273e-06,   3.18412120e-06,
         3.18412124e-06,   3.19068691e-06,   3.19162296e-06,
         3.32251561e-06,   3.37948366

(array([  4.01039181e-06,   4.01113441e-06,   4.01711403e-06,
         4.02306139e-06,   4.02306182e-06,   4.02370429e-06,
         4.02659179e-06,   4.02994723e-06,   4.03007803e-06,
         4.03432293e-06]), 10)
[  1.10000000e+01,  4.78730682e-02,  4.01039181e-06,  4.01113441e-06,
   4.01711403e-06,  4.02306139e-06,  4.02306182e-06,  4.02370429e-06,
   4.02659179e-06,  4.02994723e-06,  4.03007803e-06,  4.03432293e-06]
(array([  4.03435734e-06]), 1)
(array([  4.03435734e-06,   4.03655468e-06]), 2)
(array([  4.03435734e-06,   4.03655468e-06,   4.03931123e-06]), 3)
(array([  4.03435734e-06,   4.03655468e-06,   4.03931123e-06,
         4.04376503e-06]), 4)
(array([  4.03435734e-06,   4.03655468e-06,   4.03931123e-06,
         4.04376503e-06,   4.04376505e-06]), 5)
(array([  4.03435734e-06,   4.03655468e-06,   4.03931123e-06,
         4.04376503e-06,   4.04376505e-06,   4.04376505e-06]), 6)
(array([  4.03435734e-06,   4.03655468e-06,   4.03931123e-06,
         4.04376503e-06,   4.0437650

(array([  4.23989858e-06,   4.25215441e-06,   4.25337421e-06,
         4.25365041e-06,   4.25666959e-06,   4.26336090e-06,
         4.26628638e-06,   4.26920680e-06,   4.26920680e-06]), 9)
(array([  4.23989858e-06,   4.25215441e-06,   4.25337421e-06,
         4.25365041e-06,   4.25666959e-06,   4.26336090e-06,
         4.26628638e-06,   4.26920680e-06,   4.26920680e-06,
         4.26920680e-06]), 10)
[  1.70000000e+01,  3.92480359e-02,  4.23989858e-06,  4.25215441e-06,
   4.25337421e-06,  4.25365041e-06,  4.25666959e-06,  4.26336090e-06,
   4.26628638e-06,  4.26920680e-06,  4.26920680e-06,  4.26920680e-06]
(array([  4.26920680e-06]), 1)
(array([  4.26920680e-06,   4.28372647e-06]), 2)
(array([  4.26920680e-06,   4.28372647e-06,   4.29159524e-06]), 3)
(array([  4.26920680e-06,   4.28372647e-06,   4.29159524e-06,
         4.29478812e-06]), 4)
(array([  4.26920680e-06,   4.28372647e-06,   4.29159524e-06,
         4.29478812e-06,   4.29796373e-06]), 5)
(array([  4.26920680e-06,   4.2837264